In [2]:
# imports

import sys
sys.path.append("..")

import pandas as pd
from portfolio.greek_aggregator import aggregate_portfolio


In [3]:
#example portfolio posns

portfolio_data = pd.DataFrame([
    {"S":20000, "K":20000, "T":30/365, "r":0.06, "sigma":0.20, "option_type":"call", "quantity":1},
    {"S":20000, "K":20000, "T":30/365, "r":0.06, "sigma":0.20, "option_type":"put",  "quantity":1},
    {"S":20000, "K":21000, "T":30/365, "r":0.06, "sigma":0.20, "option_type":"call", "quantity":-1}
])


In [4]:
net_greeks = aggregate_portfolio(portfolio_data)

net_greeks

delta      -0.139493
gamma       0.000426
vega     2800.533103
theta   -3193.952148
dtype: float64

In [9]:
# testing pnl approximation

from risk.pnl_approximation import approximate_pnl

# Example shocks
dS = 200        # underlying up 200 points
dVol = 0.02     # volatility +2%
dT = -1/365     # one day decay

pnl_estimate = approximate_pnl(
    net_greeks["delta"],
    net_greeks["gamma"],
    net_greeks["vega"],
    net_greeks["theta"],
    dS, dVol, dT
)

print(pnl_estimate)

45.38086055391709


In [11]:
from risk.scenario_engine import run_scenarios

price_shocks = [-0.02, 0.0, 0.02]
vol_shocks = [-0.02, 0.0, 0.02]

scenario_results = run_scenarios(
    net_greeks,
    S=20000,
    price_shocks=price_shocks,
    vol_shocks=vol_shocks,
    time_shock=-1/365
)

scenario_results

,price_shock_%,vol_shock,estimated_pnl
0,-0.02,-0.02,42.610332
1,-0.02,0.00,98.620994
2,-0.02,0.02,154.631656
3,0.00,-0.02,-47.260108
4,0.00,0.00,8.750554
5,0.00,0.02,64.761216
6,0.02,-0.02,-68.984243
7,0.02,0.00,-12.973580
8,0.02,0.02,43.037082


In [19]:
# testing

import numpy as np

rows = []

for ps in np.linspace(-0.03, 0.03, 20):
    for vs in np.linspace(-0.03, 0.03, 20):

        dS = 20000 * ps
        dVol = vs
        dT = -1/365

        pnl = approximate_pnl(
            net_greeks["delta"],
            net_greeks["gamma"],
            net_greeks["vega"],
            net_greeks["theta"],
            dS=dS,
            dVol=dVol,
            dT=dT
        )

        rows.append({
            "delta": net_greeks["delta"] * dS,
            "gamma": 0.5 * net_greeks["gamma"] * (dS**2),
            "vega": net_greeks["vega"] * dVol,
            "theta": net_greeks["theta"] * dT,
            "pnl": pnl
        })

reg_df = pd.DataFrame(rows)


In [22]:
from analytics.risk_attribution import run_regression

model = run_regression(reg_df)
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    pnl   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.122e+33
Date:                Thu, 19 Feb 2026   Prob (F-statistic):               0.00
Time:                        11:43:12   Log-Likelihood:                 11942.
No. Observations:                 400   AIC:                        -2.388e+04
Df Residuals:                     396   BIC:                        -2.386e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
delta          1.0000   2.59e-17   3.86e+16      0.0